In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
from boardlaw.main import *
run()

In [ ]:
%debug

In [ ]:
import json
from pathlib import Path
runs = [f'mohex-{n}' for n in [3, 5, 7, 9, 11]]
for r in runs:
    js = list(sql
          .stored(r)
          .drop(['run_name', 'boardsize'], 1)
          .set_index(['black_name', 'white_name'])
          .astype(int)
          .reset_index()
          .to_dict(orient='index')
          .values())
    
    p = Path(f'output/arena/{r}.json')
    p.parent.mkdir(exist_ok=True, parents=True)
    p.write_text(json.dumps(js))

In [ ]:
a = [1, 2, 3]
a[:] = [4, 5]

In [ ]:
from boardlaw.arena import database
database.symmetric('mohex-3')

In [ ]:
!rm "output/pavlov/2020-12-29 15-44-20 fresh-wave big-buffer fine-tune/arena.json.lock"